Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Categorization and Complex System Instructions

This notebook provides an example for simplifying long or complex system instructions when using Gemini on Vertex AI. For more information please visit https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini and https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions

Import python libraries

In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
from vertexai.preview.generative_models import grounding
import json
import yaml

Set your project variables. Change "YOUR_PROJECT_ID" to your GCP project ID.

In [2]:
project_id = "rkiles_demo-host-vpc"
location = "global"
region = "us-central1"
context_file = "context.yaml"

# Define the prompt to request a recipe for a particular cuisine 

Change the folloiwng cuisine to test the categorization agent and generate a recipe. You can try items like Pizza, Chicken Cordon Bleu, 
Samosa's, or even items that span multipel different categories like Rice.

In [3]:
prompt = f'''
Show me a recipe for making Jambalaya.
'''

# Define the Categorization Agent (i.e. cat_agent)

This will use an agent to define the categorization of the provided cuisine and return the response in a JSON object. The JSON object will include both a food_category value and a food_code value. These values will be used to provide a custom context to the recipe agent.

Define the response schema. In this example we will generate values for "category" and "code" to provide speficic system instructions to the Recipe agent based on the type of cuisine. Category will be an easily readable food category and used to provide additional information to the persona for the recipe model. Code will be used to provide specific information in the context section for the recipe model. 

In [4]:
response_schema = {
    "type": "object",
    "properties": {
        "category": {
            "type": "string",
        },
        "code": {
            "type": "string",
        },
    },
    "required": ["category", "code"],
}

Define the generation configuration parameters. Here we're going to specify the response should be in JSON format and the  tempeture will be set to 0

In [5]:
cat_agent_config = GenerationConfig(
    #response_mime_type = "text/plain",
    response_mime_type = "application/json",
    temperature=0,
    #top_p=0.2,
    #top_k=2,
    #candidate_count=1,
    #max_output_tokens=8192,
    response_schema=response_schema
)

Next we will define the system instructions for the model. In this example we will instruct the model to categorize the type of cuisine to one of 7 different options. This categorization will then be used to later define the system instructions for the Recipe agent.

In [6]:
cat_agent_sys_instruct = f'''
<PERSONA>
You are a professional chef, skilled in identifying different culinary regions and cuisines
</PERSONA>

<CONTEXT>
There are 7 categories and their associated code to choose from:
Louisiana Creole: louisiana_creole
Latin American: lat_am
Caribbean: carib
Indian: indian
Western European: west_euro
Fusion: fusion
Other: other
</CONTEXT>

<OBJECTIVE>
The goal is to correctly identify the culinary style and assign it to one of the 7 listed categories.
</OBJECTIVE>
'''


In [7]:
cat_agent_model = GenerativeModel(
    "gemini-1.5-flash-002",
    generation_config=cat_agent_config,
    system_instruction=[cat_agent_sys_instruct],
)

# Generate the categorization

Generate the json data for category and code

In [8]:
cat_agent = cat_agent_model.generate_content(prompt)

In [9]:
print(cat_agent.text)

{"category": "Louisiana Creole", "code": "louisiana_creole"}


Load the json data and define the food_category and food_code variables

In [10]:
cat_agent_data = json.loads(cat_agent.text)
food_category = cat_agent_data['category']
food_code = cat_agent_data['code']

In [11]:
print(food_category)
print(food_code)

Louisiana Creole
louisiana_creole


# Define special instructions based on the identified food category

As defined in the Categorization Agent's system instructions, the available categories and their associated codes are:

Louisiana Creole: louisiana_creole

Latin American: lat_am

Caribbean: carib

Indian: indian

Western European: west_euro

Fusion: fusion

Other: other

## Load the context.yaml file

The context.yaml file contains the system context that will be used based on the identified food code.

We will first start by defining a function to load the contet of a yaml file.

In [12]:
def load_yaml_data(filepath):
    """Loads data from a YAML file"""
    try:
        with open(filepath, 'r') as f:
            data = yaml.safe_load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return None
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {e}")
        return None

Now we will call the function and load the data into context_data 

In [13]:
context_data = load_yaml_data(context_file)

Based on the identified category, the context we will be using in the Recipe agent is:

In [14]:
print(context_data[food_code])

<CONTEXT>
This cuisine is a vibrant and flavorful culinary tradition born in Louisiana, particularly in New Orleans. 
It's a melting pot of influences, reflecting the diverse cultures that shaped the region – French, Spanish, African, Native American, and even a touch of Italian and German. 
This cuisine often reflects a more urban and refined sensibility, with influences from the regions more affluent kitchens.

Key characteristics of Creole cuisine include:
The "Holy Trinity":  Onions, celery, and bell peppers form the aromatic base of countless Creole dishes, much like mirepoix in French cuisine.
Roux: A thickening agent made by cooking flour in fat, usually butter or oil. The roux can range in color from blonde (for lighter sauces and soups) to dark brown (for gumbo and stews), imparting a rich flavor and texture.
Seafood: Given Louisiana's proximity to the Gulf of Mexico, seafood plays a prominent role, including shrimp, crab, oysters, crawfish, and fish.
Tomatoes: Tomatoes are a 

# Instantiate the Recipe Agent

This agent will use the output of the categorization agent and use the specific context for the identified food category to generate a recipe.

Define the generation config parameters for the recipe agent

In [15]:
recipe_agent_config = GenerationConfig(
    response_mime_type = "text/plain",
    #response_mime_type = "application/json",
    temperature=1,
    top_p=0.5,
    top_k=3,
    #candidate_count=1,
    #max_output_tokens=8192,
    #response_schema=response_schema
)

You can see here that we are using the food_category value to guide the persona of the recipe agent and the food_code to specify the context.
This will help reduce the complexity of the context window while still allowing for a wide range of very specific instructions for the model.

In [16]:
recipe_agent_sys_instruct = f'''
<PERSONA>
You are a culinary expert, specalizing in {food_category} cuisine.
</PERSONA>

{context_data[food_code]}

<OBJECTIVE>
Based on the provided context about '{food_category}', create a delicious and authentic recipe.
</OBJECTIVE>
'''


Now we will instantiate the recipe agent and generate the requested recipe

In [17]:
recipe_agent_model = GenerativeModel(
    "gemini-1.5-pro-002",
    generation_config=recipe_agent_config,
    #tools=[tool_final],
    system_instruction=[recipe_agent_sys_instruct],
)

recipe_agent = recipe_agent_model.start_chat()

## Start with a simple chat session

We will start the recipe agent chat session with the same prompt we used for the categorization agent.

In [18]:
response = recipe_agent.send_message(prompt)
print(response.text)

## Shrimp and Andouille Sausage Jambalaya

This recipe focuses on shrimp and Andouille sausage, but you can easily adapt it with chicken, other seafood, or even rabbit.

**Yields:** 6-8 servings
**Prep time:** 20 minutes
**Cook time:** 45 minutes


**Ingredients:**

* 1 pound Andouille sausage, sliced into 1/4-inch thick rounds
* 1 pound large shrimp, peeled and deveined
* 1 cup long-grain rice
* 1 large onion, chopped
* 1 green bell pepper, chopped
* 2 celery stalks, chopped (the Holy Trinity)
* 4 cloves garlic, minced
* 1 (14.5 ounce) can diced tomatoes, undrained
* 1 (10 ounce) can diced tomatoes and green chilies (Rotel), undrained
* 4 cups chicken broth
* 1 teaspoon cayenne pepper (or more, to taste)
* 1 teaspoon smoked paprika
* 1/2 teaspoon black pepper
* 1/2 teaspoon white pepper
* 1/2 teaspoon dried thyme
* 1/4 teaspoon dried oregano
* 2 bay leaves
* 1/4 cup chopped green onions, for garnish
* Cooked white rice, for serving (optional)


**Instructions:**

1. **Brown the Sausag

## Now let's continue the chat session with the recipe agent

In [19]:
prompt = "I need to feed 10 people. How should I change the measurements?"
response = recipe_agent.send_message(prompt)
print(response.text)

To scale this Shrimp and Andouille Sausage Jambalaya recipe to feed 10 people, you'll roughly need to increase the ingredients by 1.5 times. Here's the adjusted recipe:

**Yields:** 10 servings
**Prep time:** 25 minutes
**Cook time:** 50-55 minutes

**Ingredients:**

* 1.5 pounds Andouille sausage, sliced into 1/4-inch thick rounds
* 1.5 pounds large shrimp, peeled and deveined
* 1.5 cups long-grain rice
* 1.5 large onions, chopped (or 1 very large onion)
* 1.5 green bell peppers, chopped (or 1 very large bell pepper)
* 3 celery stalks, chopped
* 6 cloves garlic, minced
* 1 (28 ounce) can diced tomatoes, undrained (or two 14.5 oz cans)
* 1 (15 ounce) can diced tomatoes and green chilies (Rotel), undrained (or one 10 oz can plus half of another)
* 6 cups chicken broth
* 1.5 teaspoons cayenne pepper (or more, to taste)
* 1.5 teaspoons smoked paprika
* 0.75 teaspoon black pepper
* 0.75 teaspoon white pepper
* 0.75 teaspoon dried thyme
* 0.375 teaspoon dried oregano (a scant 1/2 teaspoon)
